# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import geoviews as gv
import requests
import numpy as np
from citipy import citipy

# Import API key
from api_keys import geoapify_key

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,71.6,81,50,6.44,PN,2024-06-10
1,1,grand falls-windsor,48.9332,-55.6649,51.8,94,93,2.24,CA,2024-06-10
2,2,puerto natales,-51.7236,-72.4875,33.8,99,100,0.94,CL,2024-06-10
3,3,klyuchi,52.2667,79.1667,89.6,31,42,1.47,RU,2024-06-10
4,4,carloforte,39.1450,8.3058,69.8,77,14,5.21,IT,2024-06-10


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
%%capture --no-display

map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size='Humidity',
    color='blue',
    alpha=0.5,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    title='City Humidity'
)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [31]:
# Narrow down cities that fit criteria (for example, humidity > 70 and max temp < 90)
criteria_df = city_data_df[
    (city_data_df['Humidity'] > 50) &
    (city_data_df['Max Temp'] > 65) &
    (city_data_df['Max Temp'] < 90) &
    (city_data_df['Wind Speed'] < 2.5) &
    (city_data_df['Cloudiness'] == 0)
]


# Drop any rows with null values
criteria_df.dropna(inplace=True)

# Display sample data
criteria_df

C:\Users\kimmi\AppData\Local\Temp\ipykernel_22356\2470518622.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  criteria_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
152,152,stansbury park,40.6377,-112.2961,66.2,63,0,2.06,US,2024-06-10
192,192,larose,29.5724,-90.3818,75.2,91,0,0.00,US,2024-06-10
226,226,lake city,30.1897,-82.6393,68.0,95,0,1.54,US,2024-06-10
228,228,las veredas,23.1388,-109.7075,78.8,60,0,1.24,MX,2024-06-10
377,377,thomaston,32.8882,-84.3266,73.4,80,0,0.45,US,2024-06-10
431,431,heroica guaymas,27.9333,-110.9000,78.8,65,0,1.21,MX,2024-06-10
519,519,porto velho,-8.7619,-63.9039,73.4,88,0,0.00,BR,2024-06-10


### Step 3: Create a new DataFrame called `hotel_df`.

In [32]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = criteria_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
152,stansbury park,US,40.6377,-112.2961,63,
192,larose,US,29.5724,-90.3818,91,
226,lake city,US,30.1897,-82.6393,95,
228,las veredas,MX,23.1388,-109.7075,60,
377,thomaston,US,32.8882,-84.3266,80,
431,heroica guaymas,MX,27.9333,-110.9000,65,
519,porto velho,BR,-8.7619,-63.9039,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set the search radius (in meters)
radius = 10000

# Create a dictionary with search parameters
params = {
    "categories": "accommodation.hotel",  # We're searching for hotels
    "apiKey": geoapify_key                # Use your Geoapify API key
}

# Inform the user that the hotel search is starting
print("Starting hotel search")

# Loop through each row in the hotel DataFrame
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude for the current row
    lat = row["Lat"]
    lon = row["Lng"]

# Update the search parameters to include the current location
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

# Set the base URL for the Geoapify API
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request to search for hotels near the current location
    response = requests.get(base_url, params=params)

# Convert the API response to JSON format
    data = response.json()

# Try to get the first hotel's name from the response
    try:
        hotel_name = data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, use a default message
        hotel_name = "No hotel found"

# Save the hotel name in the DataFrame
    hotel_df.loc[index, "Hotel Name"] = hotel_name

# Print the search result for the current city
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Filter out rows where no hotel was found
filtered_hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]

# Add a color column to the DataFrame
filtered_hotel_df['Color'] = filtered_hotel_df['City']

# Display sample data
print(filtered_hotel_df.head())


Starting hotel search
stansbury park - nearest hotel: Comfort Inn & Suites
larose - nearest hotel: No hotel found
lake city - nearest hotel: The Blanche
las veredas - nearest hotel: Hotel Aeropuerto
thomaston - nearest hotel: Quality Inn
heroica guaymas - nearest hotel: Hotel Playa de Cortéz
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho
                City Country      Lat       Lng  Humidity  \
152   stansbury park      US  40.6377 -112.2961        63   
226        lake city      US  30.1897  -82.6393        95   
228      las veredas      MX  23.1388 -109.7075        60   
377        thomaston      US  32.8882  -84.3266        80   
431  heroica guaymas      MX  27.9333 -110.9000        65   

                Hotel Name            Color  
152   Comfort Inn & Suites   stansbury park  
226            The Blanche        lake city  
228       Hotel Aeropuerto      las veredas  
377            Quality Inn        thomaston  
431  Hotel Playa de Cortéz  heroica guaymas 

C:\Users\kimmi\AppData\Local\Temp\ipykernel_22356\510169349.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_hotel_df['Color'] = filtered_hotel_df['City']


In [34]:
# Configure and display the map plot using hvPlot
map_plot = filtered_hotel_df.hvplot.points(
    'Lng', 'Lat', geo=True, color='Color', size='Humidity', 
    tiles='OSM', hover_cols=['City'],
    title='Cities with Nearest Hotels', width=800, height=600,
    legend=True  # Enable the legend
)
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Color,Humidity,City)

In [35]:
# Display sample data
print(filtered_hotel_df.head())


                City Country      Lat       Lng  Humidity  \
152   stansbury park      US  40.6377 -112.2961        63   
226        lake city      US  30.1897  -82.6393        95   
228      las veredas      MX  23.1388 -109.7075        60   
377        thomaston      US  32.8882  -84.3266        80   
431  heroica guaymas      MX  27.9333 -110.9000        65   

                Hotel Name            Color  
152   Comfort Inn & Suites   stansbury park  
226            The Blanche        lake city  
228       Hotel Aeropuerto      las veredas  
377            Quality Inn        thomaston  
431  Hotel Playa de Cortéz  heroica guaymas  


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
%%capture --no-display
# Configure and display the map plot using hvPlot
map_plot = filtered_hotel_df.hvplot.points(
    'Lng', 'Lat', geo=True, color='Color', size='Humidity', 
    tiles='OSM', hover_cols=['City', 'Hotel Name', 'Country'],
    title='Cities with Nearest Hotels', width=800, height=600,
    legend=True  # Enable the legend
)
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Color,Humidity,City,Hotel Name,Country)